# Create novel NeuroML models from components on NeuroML-DB

This notebook demonstrates how to access the [NeuroML-DB database](https://neuroml-db.org), extract elements in NeuroML format and use them to create new models

In [1]:
from pyneuroml import pynml
import urllib.request, json 
import requests


## Create simple cell model from channels accessed via API

### Search in the database for a particular type of channel


In [2]:
types = {'cell':'NMLCL','channel':'NMLCH'}

# Helper method for search
def search_neuromldb(search_term, type=None):
    
    with urllib.request.urlopen('https://neuroml-db.org/api/search?q=%s' % search_term.replace(' ','%20')) as url:
        data = json.load(url)

    #print(data)
    for l in data:
        
        if type!=None:
            for type_ in types:
                if type==type_ and not types[type_] in l['Model_ID']:
                    data.remove(l)

        if l in data:
            print('%s: %s, %s %s'%(l['Model_ID'],l['Name'],l['First_Author'],l['Publication_Year']))
        
        
    return data

data = search_neuromldb("Fast Sodium", 'channel')

NMLCH000008: NaF Inactivating Fast Sodium, None 1998
NMLCH000023: NaF Inactivating Fast Sodium, None 1998
NMLCH000110: NaTa Fast Inactivating Sodium, None 2002
NMLCH000111: NaTs Fast Inactivating Sodium, None 2002
NMLCH000131: NaF Fast Sodium, None 2008
NMLCH000157: NaF Fast Transient Inactivating Sodium, None 2003
NMLCH000158: NaF Fast Transient Inactivating Sodium, None 2005
NMLCH000159: NaF Fast Transient Inactivating Sodium, None 2005
NMLCH000160: NaF Fast Transient Inactivating Sodium, None 2003
NMLCH000161: NaF Fast Transient Inactivating Sodium, None 2005
NMLCH000162: NaF Fast Transient Inactivating Sodium, None 2005
NMLCH000163: NaF Fast Transient Inactivating Sodium, None 2005
NMLCH000164: NaP Persistent Noninactivating Sodium, None 2003
NMLCH000165: NaF Fast Transient Inactivating Sodium, None 2005
NMLCH000166: NaF Fast Transient Inactivating Sodium, None 2005
NMLCH000167: NaF Fast Transient Inactivating Sodium, None 2005
NMLCH000168: NaF Fast Transient Inactivating Sodium, N

### Select the first of these results and download it, and browse the contents

In [3]:
# Helper method to retrieve a NeuroML file based on modelID
def get_model_from_neuromldb(model_id, type):
  
    fname = '%s.%s.nml'%(model_id, type)

    url = 'https://neuroml-db.org/render_xml_file?modelID=%s'%model_id
    r = requests.get(url)
    with open(fname , 'wb') as f:
        f.write(r.content)

    return pynml.read_neuroml2_file(fname)


# Choose the first channel
chan_model_id = data[0]['Model_ID']

na_chan_doc = get_model_from_neuromldb(chan_model_id, 'channel')

na_chan = na_chan_doc.ion_channel[0] # select the first ion channel in the nml doc

print('Channel %s has notes: %s'%(na_chan.id, na_chan.notes))

pyNeuroML >>> INFO - Loading NeuroML2 file: NMLCH000008.channel.nml


Channel Gran_NaF_98 has notes: Fast inactivating Na+ channel


### Find a Potassium channel

In [4]:
# Look at content of file
#!cat NMLCH000008.channel.nml

data = search_neuromldb("Noninactivating Potassium", 'channel')

k_chan_doc = get_model_from_neuromldb(data[0]['Model_ID'], 'channel')
k_chan = k_chan_doc.ion_channel[0] # select the first ion channel in the nml doc

print('Channel %s has notes: %s'%(k_chan.id, na_chan.notes))


/home/padraig/anaconda2/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NMLCH000113: K Fast Noninactivating Potassium, None 1992
NMLCH000128: IM Slow M Type Potassium, None 1993
NMLCH001394: K Fast Noninactivating Potassium, None 2011
NMLCH001465: K Fast Noninactivating Potassium, Berg 2018


pyNeuroML >>> INFO - Loading NeuroML2 file: NMLCH000113.channel.nml


Channel SKv3_1 has notes: Fast inactivating Na+ channel


## Create simple network model from cell accessed via API
